In [1]:
import pandas as pd
import pickle
import os
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder


In [2]:
df=pd.read_csv('data/Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.drop(columns=['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [5]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
label_encoder=LabelEncoder()

In [7]:
df['Gender']=label_encoder.fit_transform(df['Gender'])

In [8]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [10]:
onehot_encoder=OneHotEncoder()

onehot_encoder_geo=onehot_encoder.fit_transform(df[['Geography']]).toarray()

In [13]:
onehot_encoder_geo=pd.DataFrame(onehot_encoder_geo,columns=onehot_encoder.get_feature_names_out(['Geography']))

In [14]:
onehot_encoder_geo

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [15]:
df=pd.concat((df.drop('Geography',axis=1),onehot_encoder_geo),axis=1)

In [16]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [17]:
os.makedirs('regression-pickle-file')

In [18]:
with open('regression-pickle-file/onehot_encoder.pkl','wb') as file:
    pickle.dump(onehot_encoder,file)
    
with open('regression-pickle-file/label_encoder.pkl','wb') as file:
    pickle.dump(label_encoder,file)

In [19]:
x=df.drop('EstimatedSalary',axis=1)

In [20]:
x.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,0,0.0,0.0,1.0


In [21]:
y=df['EstimatedSalary']

In [22]:
scaler=StandardScaler()

In [23]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)

In [25]:
x_train_scaled

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [26]:
with open('regression-pickle-file/scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [30]:
(x_train_scaled.shape[1],)

(12,)

In [31]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train_scaled.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])

In [33]:
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from datetime import datetime

In [36]:
log_dir='regressionlogs/fit/'+ datetime.now().strftime("%Y%m%d-%H%M%S")
Tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [37]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [39]:
history=model.fit(
    x_train,y_train,
    validation_data=(x_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback,Tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 5ms/step - loss: 69786.1172 - mae: 69786.1172 - val_loss: 66425.3516 - val_mae: 66425.3516
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 62069.8125 - mae: 62069.8125 - val_loss: 56147.5273 - val_mae: 56147.5273
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 53322.7969 - mae: 53322.7969 - val_loss: 51010.5547 - val_mae: 51010.5547
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 51738.7031 - mae: 51738.7031 - val_loss: 50954.9688 - val_mae: 50954.9688
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 51362.7422 - mae: 51362.7422 - val_loss: 50527.5000 - val_mae: 50527.5000
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 51224.9922 - mae: 51224.9922 - val_loss: 50539.3281 - val_mae: 50539.3281
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 51207.2539 - mae: 51207.2539 

In [44]:
%reload_ext tensorboard


In [45]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 19900), started 0:00:53 ago. (Use '!kill 19900' to kill it.)

In [46]:
model.save('regression-pickle-file/model.h5')

e:\machine learning\project\ANN-classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
